___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />



# <font color= paletvioletred > Laboratorio 4 - Analisis Fundamental</font>




### Integrantes: 
- <Strong> Chavez Jacobo Diana Teresa </Strong> 


- <Strong> Calderón Erape Jazmín </Strong>

#### 14 Abril 2019

___
### <Strong> <font color = skyblue> I. Motivación del Laboratorio </Strong>

___
### <Strong> <font color = skyblue> II. Objetivos específicos </Strong>

In [52]:
# -- ------------------------------------------------------------- Importar modulos a utilizar -- #
# -- ------------------------------------------------------------- --------------------------- -- #

import ta as ta
import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import plotly.graph_objs as go
import plotly as py
from datetime import datetime, date, time, timedelta
import calendar

In [2]:
# -- -------------------------------------------------------------- Parametros para Data Frame -- #
# -- ------------------------------------------------------------------- --------------------- -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [3]:
# -- ------------------------------------------------------------------- Parametros para OANDA -- #
# -- ------------------------------------------------------------------- --------------------- -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

#A1_OA_Ak = '2' + '6b62ddbe404c61a1cc9da0ed8395945-52d44ef76c42b62460581783bba6c8e' + 'a'
A1_OA_Ak =  "6ff" + "6703d22fd972709992df543492afe" + "-f954a9914f9b39f72fba177a23eb243" + "b" #mi api
#F1 = "2017-04-14T12:30:00Z"
#F2 = "2019-04-10T12:30:00Z"

In [4]:
# -- -------------------------------------------------------------------- Estructuras de datos -- #
# -- ------------------------------------------------------------------- --------------------- -- #

# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, segun fechas y granularidad
# -- -- Fecha, Apertura, Maximo, Minimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrio operacion: compra/venta",
# -- -- "Se cerro operacion: Con perdia de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion maximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operacion: 1) TP o SL, 2) Regla con indicador

indice = [1, 2, 3, 4, 5]
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s

In [5]:
# -- ---------------------------------------------------------------- Inicializar API de OANDA -- #
# -- ---------------------------------------------------------------- ------------------------ -- #

api = API(access_token=A1_OA_Ak)

In [6]:
# -- ---------------------------------------------------------------- Función de fechas para descargar precios -- #
# -- ---------------------------------------------------------------- ------------------------ -- -------- -----#
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result



In [17]:
# -- ---------------------------------------------------------------- vetor de fechas por periodos de 15 dias--- #
# -- ---------------------------------------------------------------- ------------------------ -- -------- -----#

end_date = date(2019, 4, 10)
fechas = date_range(start_date, end_date, 15, 'days')

F1 = fechas[0].strftime('%Y-%m-%dT%H:%M:%S')
F2 = fechas[1].strftime('%Y-%m-%dT%H:%M:%S')

In [18]:
fechas

[datetime.date(2017, 4, 13),
 datetime.date(2017, 4, 28),
 datetime.date(2017, 5, 13),
 datetime.date(2017, 5, 28),
 datetime.date(2017, 6, 12),
 datetime.date(2017, 6, 27),
 datetime.date(2017, 7, 12),
 datetime.date(2017, 7, 27),
 datetime.date(2017, 8, 11),
 datetime.date(2017, 8, 26),
 datetime.date(2017, 9, 10),
 datetime.date(2017, 9, 25),
 datetime.date(2017, 10, 10),
 datetime.date(2017, 10, 25),
 datetime.date(2017, 11, 9),
 datetime.date(2017, 11, 24),
 datetime.date(2017, 12, 9),
 datetime.date(2017, 12, 24),
 datetime.date(2018, 1, 8),
 datetime.date(2018, 1, 23),
 datetime.date(2018, 2, 7),
 datetime.date(2018, 2, 22),
 datetime.date(2018, 3, 9),
 datetime.date(2018, 3, 24),
 datetime.date(2018, 4, 8),
 datetime.date(2018, 4, 23),
 datetime.date(2018, 5, 8),
 datetime.date(2018, 5, 23),
 datetime.date(2018, 6, 7),
 datetime.date(2018, 6, 22),
 datetime.date(2018, 7, 7),
 datetime.date(2018, 7, 22),
 datetime.date(2018, 8, 6),
 datetime.date(2018, 8, 21),
 datetime.date(201

In [175]:
type(fechas[0])

datetime.date

In [176]:
type(date(2019, 4, 10))

datetime.date

In [21]:
# -- -------------------------------------------------------------- Obtener precios historicos -- #
# -- -------------------------------------------------------------- -------------------------- -- #

params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

In [22]:
lista = []
for i in range(len(A1_Hist['candles'])-1):
        lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lista)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

___
### <Strong> <font color = skyblue> Indicador: Consumer Price Index ex Food & Energy (MoM)</Strong>

In [27]:
indicador = pd.read_csv('https://raw.githubusercontent.com/JazminErape/Lab4_eq2_AnalisisFundamental/master/Consumer%20Price%20Index%20ex%20Food%20%26%20Energy%20(MoM).csv')
indicador

,DateTime,Actual,Consensus,Previous
0,4/10/19 12:30,0.1,0.2,0.1
1,3/12/19 12:30,0.1,0.2,0.2
2,2/13/19 13:30,0.2,0.2,0.2
3,1/11/19 13:30,0.2,0.2,0.2
4,12/12/18 13:30,0.2,0.2,0.2
...,...,...,...,...
20,8/11/17 12:30,0.1,0.2,0.1
21,7/14/17 12:30,0.1,0.2,0.1
22,6/14/17 12:30,0.1,0.2,0.1
23,5/12/17 12:30,0.1,0.2,-0.1


___
### <Strong> <font color = skyblue> DataFrame1 </Strong>

In [28]:
pd_hist

,TimeStamp,Open,High,Low,Close
0,2017-04-13 04:00:00,18.55548,18.55602,18.55522,18.55556
1,2017-04-13 04:05:00,18.55554,18.56308,18.55492,18.56267
2,2017-04-13 04:10:00,18.56274,18.56502,18.56166,18.56340
3,2017-04-13 04:15:00,18.56334,18.56395,18.56307,18.56307
4,2017-04-13 04:20:00,18.56306,18.56580,18.56304,18.56424
...,...,...,...,...,...
3154,2017-04-28 03:30:00,19.06728,19.06746,19.06616,19.06740
3155,2017-04-28 03:35:00,19.06743,19.07052,19.06734,19.07014
3156,2017-04-28 03:40:00,19.07015,19.07496,19.07014,19.07486
3157,2017-04-28 03:45:00,19.07488,19.07502,19.06750,19.06783


___
### <Strong> <font color = skyblue> Escenarios </Strong>

In [30]:

for i in range(len(indicador['Actual'])):
    if indicador['Actual'][i]>= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario A
        print('A')
    elif indicador['Actual'][i]>= indicador['Consensus'][i]<=indicador['Previous'][i]:#Escenario B
        print('B')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario C
        print('c')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]<=indicador['Previous'][i]: #Escenario D
        print('D') 
    
    

c
c
A
A
A
A
c
c
A
A
A
c
A
B
B
A
c
A
c
A
c
c
c
c
c


___
### <Strong> <font color = skyblue> Ventanas </Strong>

In [322]:
def ventana(F1,F2):
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    lista = []

    for i in range(len(A1_Hist['candles'])-1):
            lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                          'Open': A1_Hist['candles'][i]['mid']['o'],
                          'High': A1_Hist['candles'][i]['mid']['h'],
                          'Low': A1_Hist['candles'][i]['mid']['l'],
                          'Close': A1_Hist['candles'][i]['mid']['c']})

    pd_hist = pd.DataFrame(lista)
    pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    return(pd_hist)

In [328]:
# -- --------------------------------------Ventana indicador 1 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2019-04-10T11:40:00Z",F2 = "2019-04-10T13:30:00Z")

,TimeStamp,Open,High,Low,Close
0,2019-04-10 11:40:00,18.85804,18.85811,18.85555,18.85556
1,2019-04-10 11:45:00,18.85558,18.85976,18.85188,18.85188
2,2019-04-10 11:50:00,18.85190,18.86140,18.85189,18.85867
3,2019-04-10 11:55:00,18.85873,18.86792,18.85686,18.86434
4,2019-04-10 12:00:00,18.86434,18.86434,18.85757,18.86193
...,...,...,...,...,...
16,2019-04-10 13:00:00,18.84570,18.84784,18.83723,18.83778
17,2019-04-10 13:05:00,18.83777,18.85524,18.83638,18.85522
18,2019-04-10 13:10:00,18.85520,18.86038,18.84754,18.86018
19,2019-04-10 13:15:00,18.86018,18.86350,18.84665,18.84949


In [335]:

# -- --------------------------------------Caracteristicas ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
caract1= pd_hist['Open'][0].astype(float)-pd_hist['Close'][0].astype(float)
caract1


AttributeError: 'str' object has no attribute 'astype'

In [334]:
pd_hist['Open'].astype(float)

0     18.85804
1     18.85558
2     18.85190
3     18.85873
4     18.86434
        ...   
14    18.84393
15    18.83354
16    18.84570
17    18.83777
18    18.85520
Name: Open, Length: 19, dtype: float64

In [336]:
pd_hist['Close'][0].astype(float)

AttributeError: 'str' object has no attribute 'astype'

In [ ]:
# -- --------------------------------------Caracteristicas ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #

for i in range(len(indicador['Actual'])):
    if indicador['Actual'][i]>= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario A
        print('A')
    elif indicador['Actual'][i]>= indicador['Consensus'][i]<=indicador['Previous'][i]:#Escenario B
        print('B')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario C
        print('c')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]<=indicador['Previous'][i]: #Escenario D
        print('D') 

In [263]:
# -- --------------------------------------Ventana indicador 2 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #

ventana(F1 = "2019-03-12T11:40:00Z",F2 = "2019-03-12T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2019-03-12 11:40:00,19.39690,19.40414,19.39490,19.40098
1,2019-03-12 11:45:00,19.40097,19.40392,19.39766,19.39986
2,2019-03-12 11:50:00,19.39986,19.40630,19.39807,19.40046
3,2019-03-12 11:55:00,19.40050,19.40230,19.39130,19.39533
4,2019-03-12 12:00:00,19.39533,19.40215,19.39161,19.40098
...,...,...,...,...,...
14,2019-03-12 12:50:00,19.28263,19.30494,19.28238,19.30260
15,2019-03-12 12:55:00,19.30267,19.30434,19.29094,19.29799
16,2019-03-12 13:00:00,19.29800,19.30629,19.29014,19.30583
17,2019-03-12 13:05:00,19.30584,19.31943,19.30556,19.31330


In [265]:
# -- --------------------------------------Ventana indicador 3 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
# 2/13/19 13:30
ventana(F1 = "2019-02-13T12:40:00Z",F2 = "2019-02-13T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2019-02-13 12:40:00,19.34028,19.34374,19.33438,19.34263
1,2019-02-13 12:45:00,19.34261,19.34407,19.33752,19.34312
2,2019-02-13 12:50:00,19.34313,19.34908,19.32358,19.32372
3,2019-02-13 12:55:00,19.32368,19.32740,19.31755,19.32586
4,2019-02-13 13:00:00,19.32590,19.34212,19.32590,19.33364
...,...,...,...,...,...
14,2019-02-13 13:50:00,19.37042,19.37311,19.35051,19.36215
15,2019-02-13 13:55:00,19.36212,19.38626,19.35948,19.38400
16,2019-02-13 14:00:00,19.38399,19.38577,19.36902,19.37218
17,2019-02-13 14:05:00,19.37218,19.40366,19.36938,19.40068


In [266]:
# -- --------------------------------------Ventana indicador 4 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
#1/11/19 13:30
ventana(F1 = "2019-01-11T12:40:00Z",F2 = "2019-01-11T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2019-01-11 12:40:00,19.10424,19.11199,19.10034,19.10044
1,2019-01-11 12:45:00,19.10032,19.10040,19.08704,19.09424
2,2019-01-11 12:50:00,19.09422,19.12246,19.09213,19.11734
3,2019-01-11 12:55:00,19.11737,19.14604,19.11700,19.13884
4,2019-01-11 13:00:00,19.13884,19.14884,19.13307,19.14762
...,...,...,...,...,...
14,2019-01-11 13:50:00,19.13874,19.14034,19.11880,19.12470
15,2019-01-11 13:55:00,19.12476,19.13455,19.12002,19.12877
16,2019-01-11 14:00:00,19.12882,19.13216,19.11280,19.12978
17,2019-01-11 14:05:00,19.12971,19.15134,19.12799,19.15134


In [267]:
# -- --------------------------------------Ventana indicador 5 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
#12/12/18 13:30
ventana(F1 = "2018-12-12T12:40:00Z",F2 = "2018-12-12T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-12-12 12:40:00,20.09926,20.10000,20.08904,20.08990
1,2018-12-12 12:45:00,20.08989,20.09506,20.08759,20.09168
2,2018-12-12 12:50:00,20.09169,20.10573,20.08711,20.09682
3,2018-12-12 12:55:00,20.09672,20.10154,20.07144,20.07148
4,2018-12-12 13:00:00,20.07147,20.07587,20.05700,20.06442
...,...,...,...,...,...
14,2018-12-12 13:50:00,20.03484,20.03552,20.01108,20.01980
15,2018-12-12 13:55:00,20.01982,20.02442,20.00938,20.01312
16,2018-12-12 14:00:00,20.01337,20.01407,19.98689,20.00079
17,2018-12-12 14:05:00,20.00083,20.02195,19.99580,20.01799


In [271]:
# -- --------------------------------------Ventana indicador 6 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-11-14T12:40:00Z",F2 = "2018-11-14T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-11-14 12:40:00,20.48220,20.48253,20.46910,20.46912
1,2018-11-14 12:45:00,20.46913,20.47050,20.45645,20.45658
2,2018-11-14 12:50:00,20.45658,20.46116,20.45142,20.45186
3,2018-11-14 12:55:00,20.45184,20.45605,20.44252,20.45332
4,2018-11-14 13:00:00,20.45330,20.45366,20.41300,20.41834
...,...,...,...,...,...
14,2018-11-14 13:50:00,20.40084,20.40880,20.37584,20.38436
15,2018-11-14 13:55:00,20.38432,20.39832,20.35461,20.39090
16,2018-11-14 14:00:00,20.39097,20.40457,20.37582,20.40104
17,2018-11-14 14:05:00,20.40101,20.41964,20.39339,20.41070


In [275]:
# -- --------------------------------------Ventana indicador 7 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-10-11T11:40:00Z",F2 = "2018-10-11T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-10-11 11:40:00,19.05988,19.06125,19.05038,19.05038
1,2018-10-11 11:45:00,19.05037,19.05037,19.03564,19.03691
2,2018-10-11 11:50:00,19.03692,19.05232,19.03542,19.05150
3,2018-10-11 11:55:00,19.05150,19.07278,19.05111,19.07228
4,2018-10-11 12:00:00,19.07229,19.08086,19.05048,19.07997
...,...,...,...,...,...
14,2018-10-11 12:50:00,19.01048,19.03948,19.00603,19.03779
15,2018-10-11 12:55:00,19.03778,19.05932,19.02308,19.05612
16,2018-10-11 13:00:00,19.05613,19.06186,19.03578,19.05592
17,2018-10-11 13:05:00,19.05596,19.06149,19.02820,19.03356


In [277]:
# -- --------------------------------------Ventana indicador 8 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-09-13T11:40:00Z",F2 = "2018-09-13T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-09-13 11:40:00,18.92349,18.93834,18.92332,18.93458
1,2018-09-13 11:45:00,18.93454,18.94814,18.93451,18.94814
2,2018-09-13 11:50:00,18.94816,18.96112,18.94787,18.95454
3,2018-09-13 11:55:00,18.95450,18.96404,18.95098,18.96062
4,2018-09-13 12:00:00,18.96060,18.97282,18.94880,18.95102
...,...,...,...,...,...
14,2018-09-13 12:50:00,18.85731,18.85762,18.84178,18.84874
15,2018-09-13 12:55:00,18.84887,18.85450,18.84346,18.85437
16,2018-09-13 13:00:00,18.85432,18.87428,18.84725,18.87246
17,2018-09-13 13:05:00,18.87250,18.88873,18.86875,18.87676


In [279]:
# -- --------------------------------------Ventana indicador 9 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-08-10T11:40:00Z",F2 = "2018-08-10T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-08-10 11:40:00,18.92151,18.92151,18.87366,18.88304
1,2018-08-10 11:45:00,18.88304,18.89365,18.86919,18.88619
2,2018-08-10 11:50:00,18.88622,18.92595,18.88180,18.92380
3,2018-08-10 11:55:00,18.92382,18.93502,18.92025,18.92716
4,2018-08-10 12:00:00,18.92717,18.96080,18.91525,18.95818
...,...,...,...,...,...
14,2018-08-10 12:50:00,18.89689,18.90048,18.87530,18.89830
15,2018-08-10 12:55:00,18.89826,18.91420,18.89298,18.91335
16,2018-08-10 13:00:00,18.91343,18.91343,18.90142,18.91006
17,2018-08-10 13:05:00,18.91011,18.92609,18.91011,18.92214


In [282]:
# -- --------------------------------------Ventana indicador 10 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-07-12T11:40:00Z",F2 = "2018-07-12T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-07-12 11:40:00,18.97283,18.97554,18.95576,18.95840
1,2018-07-12 11:45:00,18.95851,18.96792,18.95647,18.95888
2,2018-07-12 11:50:00,18.95888,18.96602,18.95479,18.95714
3,2018-07-12 11:55:00,18.95717,18.95728,18.93788,18.93863
4,2018-07-12 12:00:00,18.93861,18.94094,18.92574,18.94078
...,...,...,...,...,...
14,2018-07-12 12:50:00,18.85670,18.86914,18.85524,18.86832
15,2018-07-12 12:55:00,18.86834,18.87685,18.83646,18.84337
16,2018-07-12 13:00:00,18.84334,18.85595,18.82045,18.82126
17,2018-07-12 13:05:00,18.82122,18.83722,18.79824,18.81626


In [284]:
# -- --------------------------------------Ventana indicador 11 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-06-12T11:40:00Z",F2 = "2018-06-12T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-06-12 11:40:00,20.61192,20.61542,20.60882,20.61535
1,2018-06-12 11:45:00,20.61558,20.62812,20.61424,20.62553
2,2018-06-12 11:50:00,20.62554,20.62862,20.62308,20.62690
3,2018-06-12 11:55:00,20.62696,20.62708,20.60092,20.60250
4,2018-06-12 12:00:00,20.60243,20.61222,20.60170,20.60612
...,...,...,...,...,...
14,2018-06-12 12:50:00,20.55487,20.55544,20.52399,20.53350
15,2018-06-12 12:55:00,20.53351,20.55531,20.53160,20.54818
16,2018-06-12 13:00:00,20.54836,20.56930,20.54836,20.56840
17,2018-06-12 13:05:00,20.56842,20.57686,20.55963,20.56312


In [286]:
# -- --------------------------------------Ventana indicador 12 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-05-10T11:40:00Z",F2 = "2018-05-10T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-05-10 11:40:00,19.41186,19.41264,19.39594,19.40778
1,2018-05-10 11:45:00,19.40776,19.41430,19.40192,19.41415
2,2018-05-10 11:50:00,19.41416,19.43664,19.40641,19.43346
3,2018-05-10 11:55:00,19.43348,19.44106,19.42920,19.44004
4,2018-05-10 12:00:00,19.44002,19.45370,19.43860,19.45132
...,...,...,...,...,...
14,2018-05-10 12:50:00,19.36522,19.36964,19.34988,19.35276
15,2018-05-10 12:55:00,19.35274,19.35313,19.32904,19.34190
16,2018-05-10 13:00:00,19.34204,19.34410,19.32822,19.33460
17,2018-05-10 13:05:00,19.33458,19.33829,19.29966,19.31180


In [288]:
# -- --------------------------------------Ventana indicador 13 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-04-11T11:40:00Z",F2 = "2018-04-11T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-04-11 11:40:00,18.29244,18.29298,18.27500,18.27507
1,2018-04-11 11:45:00,18.27505,18.27869,18.25684,18.26126
2,2018-04-11 11:50:00,18.26128,18.27850,18.26123,18.27798
3,2018-04-11 11:55:00,18.27800,18.28245,18.27016,18.27036
4,2018-04-11 12:00:00,18.27038,18.29369,18.27034,18.29084
...,...,...,...,...,...
14,2018-04-11 12:50:00,18.26927,18.27150,18.26684,18.26896
15,2018-04-11 12:55:00,18.26894,18.27228,18.25632,18.26562
16,2018-04-11 13:00:00,18.26564,18.26840,18.25855,18.26444
17,2018-04-11 13:05:00,18.26426,18.26654,18.25854,18.26648


In [290]:
# -- --------------------------------------Ventana indicador 14 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-03-13T11:40:00Z",F2 = "2018-03-13T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-03-13 11:40:00,18.60028,18.60466,18.59896,18.60459
1,2018-03-13 11:45:00,18.60460,18.60710,18.60084,18.60350
2,2018-03-13 11:50:00,18.60346,18.60410,18.59592,18.59678
3,2018-03-13 11:55:00,18.59682,18.60084,18.58375,18.58973
4,2018-03-13 12:00:00,18.58972,18.59024,18.56619,18.56955
...,...,...,...,...,...
14,2018-03-13 12:50:00,18.51774,18.52879,18.51730,18.52768
15,2018-03-13 12:55:00,18.52766,18.53984,18.52610,18.52966
16,2018-03-13 13:00:00,18.52969,18.54005,18.52684,18.52930
17,2018-03-13 13:05:00,18.52932,18.53412,18.51548,18.51864


In [292]:
# -- --------------------------------------Ventana indicador 15 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-02-14T11:40:00Z",F2 = "2018-02-14T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-02-14 11:40:00,18.58030,18.58112,18.57344,18.57442
1,2018-02-14 11:45:00,18.57441,18.58138,18.57418,18.57792
2,2018-02-14 11:50:00,18.57791,18.58482,18.57696,18.58049
3,2018-02-14 11:55:00,18.58051,18.58879,18.58051,18.58534
4,2018-02-14 12:00:00,18.58534,18.59434,18.58200,18.58910
...,...,...,...,...,...
14,2018-02-14 12:50:00,18.60706,18.60928,18.60370,18.60416
15,2018-02-14 12:55:00,18.60417,18.61050,18.60307,18.60622
16,2018-02-14 13:00:00,18.60622,18.62120,18.60450,18.61110
17,2018-02-14 13:05:00,18.61111,18.61812,18.60018,18.60287


In [296]:
# -- --------------------------------------Ventana indicador 16 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2018-01-10T12:40:00Z",F2 = "2018-01-12T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2018-01-10 12:40:00,19.27872,19.29372,19.27036,19.27060
1,2018-01-10 12:45:00,19.27060,19.28655,19.26800,19.28655
2,2018-01-10 12:50:00,19.28656,19.30062,19.28522,19.29951
3,2018-01-10 12:55:00,19.29956,19.30692,19.29374,19.30169
4,2018-01-10 13:00:00,19.30167,19.30324,19.29164,19.29544
...,...,...,...,...,...
589,2018-01-12 13:50:00,19.12778,19.13625,19.10985,19.11661
590,2018-01-12 13:55:00,19.11664,19.13834,19.11408,19.13809
591,2018-01-12 14:00:00,19.13818,19.14786,19.12594,19.13632
592,2018-01-12 14:05:00,19.13626,19.13626,19.11648,19.12253


In [301]:
# -- --------------------------------------Ventana indicador 17 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-12-13T12:40:00Z",F2 = "2017-12-13T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-12-13 12:40:00,19.19722,19.20619,19.19720,19.20580
1,2017-12-13 12:45:00,19.20580,19.20588,19.19446,19.19599
2,2017-12-13 12:50:00,19.19604,19.20910,19.19217,19.20683
3,2017-12-13 12:55:00,19.20690,19.20796,19.20300,19.20637
4,2017-12-13 13:00:00,19.20634,19.21572,19.20176,19.20970
...,...,...,...,...,...
14,2017-12-13 13:50:00,19.15854,19.15986,19.14737,19.15364
15,2017-12-13 13:55:00,19.15366,19.15886,19.14256,19.15865
16,2017-12-13 14:00:00,19.15866,19.17092,19.15110,19.16978
17,2017-12-13 14:05:00,19.16984,19.18144,19.16225,19.17982


In [304]:
# -- --------------------------------------Ventana indicador 18 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-11-15T12:40:00Z",F2 = "2017-11-15T14:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-11-15 12:40:00,19.17432,19.18882,19.17217,19.18615
1,2017-11-15 12:45:00,19.18622,19.18746,19.18024,19.18376
2,2017-11-15 12:50:00,19.18375,19.18449,19.17870,19.18141
3,2017-11-15 12:55:00,19.18140,19.18281,19.16386,19.16488
4,2017-11-15 13:00:00,19.16490,19.16492,19.15526,19.16034
...,...,...,...,...,...
14,2017-11-15 13:50:00,19.11516,19.13016,19.11516,19.12697
15,2017-11-15 13:55:00,19.12698,19.13670,19.11174,19.11176
16,2017-11-15 14:00:00,19.11179,19.13020,19.11169,19.12775
17,2017-11-15 14:05:00,19.12774,19.14322,19.12774,19.12992


In [306]:
# -- --------------------------------------Ventana indicador 19 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-10-13T11:40:00Z",F2 = "2017-10-13T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-10-13 11:40:00,18.96444,18.97806,18.95747,18.95961
1,2017-10-13 11:45:00,18.95962,18.96222,18.94706,18.95118
2,2017-10-13 11:50:00,18.95126,18.95956,18.95126,18.95615
3,2017-10-13 11:55:00,18.95605,18.96413,18.95179,18.95684
4,2017-10-13 12:00:00,18.95682,18.96272,18.94684,18.95174
...,...,...,...,...,...
14,2017-10-13 12:50:00,18.90462,18.91499,18.88788,18.89136
15,2017-10-13 12:55:00,18.89132,18.91084,18.88694,18.89946
16,2017-10-13 13:00:00,18.89945,18.89946,18.88348,18.89606
17,2017-10-13 13:05:00,18.89602,18.89948,18.88754,18.89736


In [309]:
# -- --------------------------------------Ventana indicador 20 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-09-14T11:40:00Z",F2 = "2017-09-14T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-09-14 11:40:00,17.76272,17.77664,17.76246,17.77658
1,2017-09-14 11:45:00,17.77659,17.77694,17.76960,17.77356
2,2017-09-14 11:50:00,17.77356,17.78967,17.77318,17.78048
3,2017-09-14 11:55:00,17.78050,17.78527,17.77826,17.78281
4,2017-09-14 12:00:00,17.78279,17.78286,17.77138,17.77156
...,...,...,...,...,...
14,2017-09-14 12:50:00,17.78052,17.79398,17.77104,17.77104
15,2017-09-14 12:55:00,17.77103,17.79058,17.76530,17.78728
16,2017-09-14 13:00:00,17.78730,17.79332,17.77893,17.78176
17,2017-09-14 13:05:00,17.78180,17.80978,17.77506,17.80790


In [310]:
# -- --------------------------------------Ventana indicador 21 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-09-14T11:40:00Z",F2 = "2017-09-14T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-09-14 11:40:00,17.76272,17.77664,17.76246,17.77658
1,2017-09-14 11:45:00,17.77659,17.77694,17.76960,17.77356
2,2017-09-14 11:50:00,17.77356,17.78967,17.77318,17.78048
3,2017-09-14 11:55:00,17.78050,17.78527,17.77826,17.78281
4,2017-09-14 12:00:00,17.78279,17.78286,17.77138,17.77156
...,...,...,...,...,...
14,2017-09-14 12:50:00,17.78052,17.79398,17.77104,17.77104
15,2017-09-14 12:55:00,17.77103,17.79058,17.76530,17.78728
16,2017-09-14 13:00:00,17.78730,17.79332,17.77893,17.78176
17,2017-09-14 13:05:00,17.78180,17.80978,17.77506,17.80790


In [312]:
# -- --------------------------------------Ventana indicador 22 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-07-14T11:40:00Z",F2 = "2017-07-14T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-07-14 11:40:00,17.67650,17.67930,17.67650,17.67846
1,2017-07-14 11:45:00,17.67845,17.67891,17.67710,17.67811
2,2017-07-14 11:50:00,17.67813,17.67877,17.66835,17.66944
3,2017-07-14 11:55:00,17.66942,17.67280,17.66572,17.66590
4,2017-07-14 12:00:00,17.66590,17.66856,17.65786,17.65817
...,...,...,...,...,...
14,2017-07-14 12:50:00,17.56527,17.57460,17.56307,17.57025
15,2017-07-14 12:55:00,17.57028,17.57115,17.54429,17.54698
16,2017-07-14 13:00:00,17.54690,17.56627,17.54686,17.56412
17,2017-07-14 13:05:00,17.56420,17.56561,17.54669,17.55025


In [314]:
# -- --------------------------------------Ventana indicador 23 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-06-14T11:40:00Z",F2 = "2017-06-14T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-06-14 11:40:00,18.02732,18.03060,18.02214,18.02534
1,2017-06-14 11:45:00,18.02535,18.03808,18.02534,18.03724
2,2017-06-14 11:50:00,18.03728,18.04902,18.03701,18.04519
3,2017-06-14 11:55:00,18.04518,18.04742,18.03684,18.04718
4,2017-06-14 12:00:00,18.04710,18.04798,18.04053,18.04064
...,...,...,...,...,...
14,2017-06-14 12:50:00,17.95690,17.97062,17.95358,17.96818
15,2017-06-14 12:55:00,17.96816,17.97118,17.96200,17.96651
16,2017-06-14 13:00:00,17.96650,17.96908,17.94154,17.94872
17,2017-06-14 13:05:00,17.94874,17.96175,17.94580,17.95886


In [316]:
# -- --------------------------------------Ventana indicador 24 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-05-12T11:40:00Z",F2 = "2017-05-12T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-05-12 11:40:00,18.88232,18.89447,18.88094,18.88714
1,2017-05-12 11:45:00,18.88716,18.89674,18.88658,18.89664
2,2017-05-12 11:50:00,18.89660,18.90052,18.89045,18.89368
3,2017-05-12 11:55:00,18.89366,18.89366,18.88742,18.89132
4,2017-05-12 12:00:00,18.89130,18.89130,18.88404,18.88550
...,...,...,...,...,...
14,2017-05-12 12:50:00,18.76982,18.76985,18.73645,18.74445
15,2017-05-12 12:55:00,18.74447,18.74814,18.73217,18.74814
16,2017-05-12 13:00:00,18.74818,18.75681,18.73543,18.75295
17,2017-05-12 13:05:00,18.75294,18.76246,18.74588,18.75977


In [317]:
indicador['DateTime'][24]

'4/14/17 12:30'

In [318]:
# -- --------------------------------------Ventana indicador 25 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
ventana(F1 = "2017-04-14T11:40:00Z",F2 = "2017-04-14T13:20:00Z")

,TimeStamp,Open,High,Low,Close
0,2017-04-14 11:40:00,18.55656,18.55688,18.55345,18.55348
1,2017-04-14 11:45:00,18.55355,18.55884,18.55355,18.55884
2,2017-04-14 11:50:00,18.55882,18.55886,18.55846,18.55859
3,2017-04-14 11:55:00,18.55854,18.55934,18.55846,18.55932
4,2017-04-14 12:00:00,18.55934,18.55968,18.55904,18.55946
...,...,...,...,...,...
14,2017-04-14 12:50:00,18.50572,18.50572,18.50212,18.50349
15,2017-04-14 12:55:00,18.50342,18.50426,18.50110,18.50226
16,2017-04-14 13:00:00,18.50225,18.50595,18.50225,18.50331
17,2017-04-14 13:05:00,18.50330,18.51176,18.50211,18.51062
